In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
!pip3 install timm transformers

In [ ]:
import os
import numpy as np
import h5py
import copy

from tqdm import tqdm
import timm

from transformers import AutoImageProcessor, ViTForImageClassification

import torch
import torch.nn as nna
from torch.utils.data import Dataset, DataLoader, SequentialSampler
from torch import optim

from torchvision import transforms

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error

np.random.seed(0)
torch.manual_seed(0)

In [ ]:
features_filepath = '/content/drive/MyDrive/LakeRegression/340_Veri_toplam_temiz.xlsx'
feats_df = pd.read_excel(features_filepath)
selected_feature = 'Klorofil-a (µg/L)'
feats_df = feats_df[['Date', 'Station', selected_feature, 'X', 'Y']]
feats_df.head()

,Date,Station,Klorofil-a (µg/L),X,Y
0,2017-04-27,1,86.14,235,537
1,2017-04-27,2,61.24,280,427
2,2017-04-27,3,48.40,325,340
3,2017-04-27,4,39.70,345,263
4,2017-04-27,5,72.52,398,165


In [ ]:
scaler = StandardScaler()
feats_df[selected_feature] = scaler.fit_transform(feats_df[selected_feature].values.reshape(-1, 1))
feats_df.head()

,Date,Station,Klorofil-a (µg/L),X,Y
0,2017-04-27,1,2.643996,235,537
1,2017-04-27,2,1.361115,280,427
2,2017-04-27,3,0.699582,325,340
3,2017-04-27,4,0.251347,345,263
4,2017-04-27,5,1.942276,398,165


In [ ]:
from scipy.ndimage import zoom


desired_size = 224

def pad_image(image, desired_size):
    pad_width = (desired_size - image.shape[-1]) // 2

    if image.shape[-1] % 2 != 0:
        pad_width_1 = pad_width
        pad_width_2 = pad_width + 1
    else:
        pad_width_1 = pad_width
        pad_width_2 = pad_width

    padded_image = np.pad(image, ((0, 0), (pad_width_1, pad_width_2), (pad_width_1, pad_width_2)))
    return padded_image


def resize_image(image, desired_size):
    scale_factors = [desired_size / dim for dim in image.shape[1:]]
    scale_factors = [1] + scale_factors
    resized_image = zoom(image, scale_factors)
    return resized_image

In [ ]:
unique_dates = feats_df['Date'].unique()

use_padding = False
use_resize = True

X_train = []
y_train = []
X_val = []
y_val = []
X_test = []
y_test = []
patch_size = 3
for i in tqdm(range(1, 35)):
    if i != 22 and i != 23:
        folder_path = f'/content/drive/MyDrive/LakeRegression/data/{i}'
        for j in range(10):
            file_path = f'{folder_path}/station_{j}_{patch_size}.h5'
            if os.path.isfile(file_path):
                with h5py.File(file_path, 'r') as f:
                    a_group_key = list(f.keys())[0]
                    image = np.array(f[a_group_key])

                    unique_date = unique_dates[i - 1]
                    unique_station = j + 1
                    label = feats_df.loc[(feats_df['Date'] == unique_date) & (feats_df['Station'] == unique_station)][selected_feature].values[0]

                    date = pd.to_datetime(unique_date)
                    if date < pd.to_datetime('2019-03-15'):
                        X_train.append(image)
                        y_train.append(label)
                    elif date >= pd.to_datetime('2019-03-15') and date < pd.to_datetime('2019-05-01'):
                        X_val.append(image)
                        y_val.append(label)
                    else:
                        X_test.append(image)
                        y_test.append(label)

print(f'Train % {len(X_train) / (len(X_train) + len(X_val) + len(X_test))} | Val % {len(X_val) / (len(X_train) + len(X_val) + len(X_test))} | Test % {len(X_test) / (len(X_train) + len(X_val) + len(X_test))}')

if use_padding:
  X_train = np.array([pad_image(x, desired_size) for x in X_train])
  X_val = np.array([pad_image(x, desired_size) for x in X_val])
  X_test = np.array([pad_image(x, desired_size) for x in X_test])
  y_train = np.array(y_train)
  y_val = np.array(y_val)
  y_test = np.array(y_test)
elif use_resize:
  X_train = np.array([resize_image(x, desired_size) for x in X_train])
  X_val = np.array([resize_image(x, desired_size) for x in X_val])
  X_test = np.array([resize_image(x, desired_size) for x in X_test])
  y_train = np.array(y_train)
  y_val = np.array(y_val)
  y_test = np.array(y_test)

print(f'Train shape: {X_train.shape} | {y_train.shape} | Val shape: {X_val.shape} | {y_val.shape} | Test shape: {X_test.shape} | {y_test.shape}')

100%|██████████| 34/34 [00:01<00:00, 33.88it/s]


Train % 0.8125 | Val % 0.09375 | Test % 0.09375
Train shape: (260, 12, 224, 224) | (260,) | Val shape: (30, 12, 224, 224) | (30,) | Test shape: (30, 12, 224, 224) | (30,)


In [ ]:
class LakeDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X.astype(np.float32)
        self.y = y.astype(np.float32)
        self.transform = transform

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        image = self.X[idx]
        label = self.y[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

means = []
stds = []
for i in range(12):
    means.append(np.mean(X_train[:, i, :, :]))
    stds.append(np.std(X_train[:, i, :, :]))

transform = transforms.Compose([
    transforms.Lambda(lambda x: torch.from_numpy(x).float()),
    transforms.Normalize(means, stds)
])

train_dataset = LakeDataset(X_train, y_train, transform=transform)
val_dataset = LakeDataset(X_val, y_val, transform=transform)
test_dataset = LakeDataset(X_test, y_test, transform=transform)

# use sequential sampler to preserve the date order
train_loader = DataLoader(train_dataset, batch_size=32, sampler=SequentialSampler(train_dataset))
val_loader = DataLoader(val_dataset, batch_size=32, sampler=SequentialSampler(val_dataset))
test_loader = DataLoader(test_dataset, batch_size=32, sampler=SequentialSampler(test_dataset))

print(f'Train loader: {len(train_loader)} | Val loader: {len(val_loader)} | Test loader: {len(test_loader)}')

Train loader: 9 | Val loader: 1 | Test loader: 1


In [ ]:
class ViTBase16(nn.Module):
    def __init__(self, n_classes):

        super(ViTBase16, self).__init__()

        self.model = timm.create_model("vit_base_patch16_224", pretrained=True, in_chans=12)

        self.model.head = nn.Linear(self.model.head.in_features, n_classes)

    def forward(self, x):
        x = self.model(x)
        return x

    def train_one_epoch(self, train_loader, criterion, optimizer, device):
        epoch_loss = 0.0
        epoch_r2 = 0.0
        epoch_mse = 0.0
        epoch_rmse = 0.0

        self.model.train()
        for i, (data, target) in enumerate(train_loader):
            if device.type == "cuda":
                data, target = data.cuda(), target.cuda()
            elif device.type == "mps":
                data, target = data.to(device), target.to(device)
            elif device.type == "cpu":
                data, target = data.cpu(), target.cpu()

            optimizer.zero_grad()

            output = self.forward(data)

            loss = criterion(output, target)
            loss.backward()

            r2 = r2_score(target.cpu().detach().numpy(), output.cpu().detach().numpy())
            mse = mean_squared_error(target.cpu().detach().numpy(), output.cpu().detach().numpy())
            rmse = np.sqrt(mse)

            epoch_loss += loss
            epoch_r2 += r2
            epoch_mse += mse
            epoch_rmse += rmse

            optimizer.step()

        loss = epoch_loss / len(train_loader)
        r2 = epoch_r2 / len(train_loader)
        mse = epoch_mse / len(train_loader)
        rmse = epoch_rmse / len(train_loader)

        return loss, r2, mse, rmse

    def validate_one_epoch(self, valid_loader, criterion, device):
        valid_loss = 0.0
        valid_r2 = 0.0
        valid_mse = 0.0
        valid_rmse = 0.0

        self.model.eval()
        for data, target in valid_loader:
            if device.type == "cuda":
                data, target = data.cuda(), target.cuda()
            elif device.type == "mps":
                data, target = data.to(device), target.to(device)
            elif device.type == "cpu":
                data, target = data.cpu(), target.cpu()

            with torch.no_grad():
                output = self.model(data)
                loss = criterion(output, target)

                r2 = r2_score(target.cpu().detach().numpy(), output.cpu().detach().numpy())
                mse = mean_squared_error(target.cpu().detach().numpy(), output.cpu().detach().numpy())
                rmse = np.sqrt(mse)

                valid_loss += loss
                valid_r2 += r2
                valid_mse += mse
                valid_rmse += rmse

        loss = valid_loss / len(valid_loader)
        r2 = valid_r2 / len(valid_loader)
        mse = valid_mse / len(valid_loader)
        rmse = valid_rmse / len(valid_loader)

        return loss, r2, mse, rmse

if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

In [ ]:
import warnings
warnings.filterwarnings("ignore")

def train(model, train_loader, val_loader, criterion, optimizer, scheduler, device, epochs):
    best_loss = np.inf
    best_r2 = -np.inf
    best_mse = np.inf
    best_rmse = np.inf
    best_model = None

    for epoch in range(epochs):
        print(f'Epoch {epoch + 1} / {epochs}')
        print('-' * 10)

        train_loss, train_r2, train_mse, train_rmse = model.train_one_epoch(train_loader, criterion, optimizer, device)
        print(f'Train loss: {train_loss} | Train R2: {train_r2} | Train MSE: {train_mse} | Train RMSE: {train_rmse}')

        val_loss, val_r2, val_mse, val_rmse = model.validate_one_epoch(val_loader, criterion, device)
        print(f'Val loss: {val_loss} | Val R2: {val_r2} | Val MSE: {val_mse} | Val RMSE: {val_rmse}')
        print('-' * 10)

        scheduler.step(val_loss)

        if val_loss < best_loss:
            best_loss = val_loss
            best_r2 = val_r2
            best_mse = val_mse
            best_rmse = val_rmse
            best_model = copy.deepcopy(model)

    print(f'Best loss: {best_loss} | Best R2: {best_r2} | Best MSE: {best_mse} | Best RMSE: {best_rmse}')
    return best_model

model = ViTBase16(1)
model.to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

model = train(model, train_loader, val_loader, criterion, optimizer, scheduler, device, epochs=125)

Epoch 1 / 125
----------
Train loss: 107.29401397705078 | Train R2: -318.80994378839637 | Train MSE: 107.9126071876122 | Train RMSE: 7.843710190720028
Val loss: 1.4339667558670044 | Val R2: -12.66269795013437 | Val MSE: 1.4042556285858154 | Val RMSE: 1.185012936592102
----------
Epoch 2 / 125
----------
Train loss: 5.355320453643799 | Train R2: -42.07054336912409 | Train MSE: 5.411228193177117 | Train RMSE: 1.9677063491609361
Val loss: 1.9251153469085693 | Val R2: -17.551258048472583 | Val MSE: 1.9067028760910034 | Val RMSE: 1.3808341026306152
----------
Epoch 3 / 125
----------
Train loss: 2.6169605255126953 | Train R2: -51.476029845051016 | Train MSE: 2.418165382411745 | Train RMSE: 1.4038830796877544
Val loss: 2.900723457336426 | Val R2: -26.927552568661273 | Val MSE: 2.870400905609131 | Val RMSE: 1.694225788116455
----------
Epoch 4 / 125
----------
Train loss: 3.4589955806732178 | Train R2: -182.77229043242812 | Train MSE: 3.133231851789686 | Train RMSE: 1.6598583393626742
Val los

In [ ]:
def test(model, test_loader, criterion, device):
    test_loss, test_r2, test_mse, test_rmse = model.validate_one_epoch(test_loader, criterion, device)
    print(f'Test loss: {test_loss} | Test R2: {test_r2} | Test MSE: {test_mse} | Test RMSE: {test_rmse}')

test(model, test_loader, criterion, device)